In [ ]:
!pip install -Uqq pyts fastbook pandas-ta

import datetime as dt
import os
import warnings
import numpy as np
import pandas as pd
import pywt
import pywt.data
import re
import gc
import pandas_ta as ta
from pyts.image import GramianAngularField
from PIL import Image as im
import matplotlib.image

import fastbook
fastbook.setup_book()
from fastbook import *

In [ ]:


SYMBOL = 'BTCUSDT'
INTERVAL = '1m'
INPUT_SIZE = 30
RAW_INPUT_SIZE = 100
THRESHOLD = 0.97
TRADING_PERCENT = 0.1
INITIAL_USD_BALANCE = 1000
STOP_PROFIT = 0.004
STOP_LOSS = 0.004
ORDER_LIFE = 15

df = pd.read_csv("../input/binance-1m-v2/binance.csv", header=0,
                 names=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'].apply(lambda x: pd.to_datetime(x))

def calc_label(dataset, position):
    price = dataset['close'].to_numpy()[position]
    stop_loss = STOP_LOSS * price
    stop_profit = STOP_PROFIT * price
    label = "wait"
    bearish_stop_loss = price + stop_loss
    bearish_stop_profit = price - stop_profit
    bullish_stop_loss = price - stop_loss
    bullish_stop_profit = price + stop_profit
    for i in range(1, ORDER_LIFE+1):
        max_price = dataset['high'].to_numpy()[position+i]
        low_price = dataset['low'].to_numpy()[position+i]
        if low_price > bullish_stop_loss:
            if max_price >= bullish_stop_profit:
                label = "buy"
                break
        else:
            break
    for i in range(1, ORDER_LIFE+1):
        max_price = dataset['high'].to_numpy()[position+i]
        low_price = dataset['low'].to_numpy()[position+i]
        if max_price < bearish_stop_loss:
            if low_price <= bearish_stop_profit:
                label = "sell"
                break
        else:
            break
    return label


L = len(df['close'])
df_labels = [None]*L
for i in range(0, L-ORDER_LIFE):
    df_labels[i] = calc_label(df, i)

df['label'] = df_labels

df = df.iloc[:-ORDER_LIFE].reset_index(drop=True).copy()
df.set_index(pd.DatetimeIndex(df["timestamp"]), inplace=True)

ind_list = ['qstick', 't3', 'cti', 'mad', 'ha', 'squeeze', 'aroon', 'bbands', 'kc', 'vwap', 'stoch']

for indi in ind_list:
    print(indi)
    indi_fn = getattr(df.ta, indi)
    indi_fn(append=True)
    
df = df.iloc[RAW_INPUT_SIZE:].reset_index(drop=True).copy()
gc.collect

pat = r'^(.*)_\d+.png'
images_path = '/kaggle/working/images/'
if not os.path.exists(images_path):
    os.makedirs(images_path)
files = get_image_files(images_path)
for f in files:
    os.remove(f)
files = get_image_files(images_path)

L = len(df['label'])


In [ ]:
gaf_transformer = GramianAngularField(method='difference', image_size=INPUT_SIZE)
ind_columns = ['QS_10', 'T3_10_0.7', 'CTI_12', 'MAD_30', 'HA_low', 'SQZ_20_2.0_20_1.5', 'AROONU_14', 'BBU_5_2.0', 'KCBe_20_2', 'VWAP_D', 'STOCHd_14_3_3']

for col_name in ind_columns:
    df[col_name].replace([np.inf, -np.inf], np.nan, inplace=True)
    df[col_name].fillna(value=0, inplace=True)

for i in range(INPUT_SIZE-1, L):
    if df['label'][i] != 'wait':
        i_open = gaf_transformer.fit_transform(df['open'][i+1-INPUT_SIZE:i+1].to_numpy().reshape(1, -1)).squeeze()
        i_high = gaf_transformer.fit_transform(df['high'][i+1-INPUT_SIZE:i+1].to_numpy().reshape(1, -1)).squeeze()
        i_low = gaf_transformer.fit_transform(df['low'][i+1-INPUT_SIZE:i+1].to_numpy().reshape(1, -1)).squeeze()
        i_close = gaf_transformer.fit_transform(df['close'][i+1-INPUT_SIZE:i+1].to_numpy().reshape(1, -1)).squeeze()
        i_volume = gaf_transformer.fit_transform(df['volume'][i+1-INPUT_SIZE:i+1].to_numpy().reshape(1, -1)).squeeze()
        inputs_list = [i_open, i_high, i_low, i_close, i_volume] + [gaf_transformer.fit_transform(df[col_name][i+1-INPUT_SIZE:i+1].to_numpy().reshape(1, -1)).squeeze() for col_name in ind_columns]
        rows_list = [inputs_list[i:i + 4] for i in range(0, len(inputs_list), 4)]
        image_rows = [np.concatenate(row) for row in rows_list]
        image = np.concatenate(image_rows, axis=1)
        label = df['label'].to_numpy()[i]
        matplotlib.image.imsave(images_path + label + '_' + str(i) + '.png', image)
        gc.collect()

In [ ]:
images_path = '/kaggle/working/images/'
pat = r'^(.*)_\d+.png'
files = get_image_files(images_path)
dls = ImageDataLoaders.from_name_re(images_path, files, pat)
#dls.show_batch()
learn = vision_learner(dls, resnet18, metrics=error_rate)

In [ ]:
#learn.lr_find()

In [ ]:
learn.fine_tune(10, 0.0003981071640737355) #resnet34 the best so far

In [ ]:
learn.export(fname='/kaggle/working/model.pkl')
